In [1]:
import ctypes
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import subprocess
import torch
from functools import singledispatch, singledispatchmethod, wraps
from inspect import getcallargs
from scipy.spatial.distance import cdist
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric import nn as gnn
from torch_geometric.nn.conv import GCNConv, SimpleConv, SGConv
from tqdm import tqdm
from asterism.core import buildmethod, checkmethod, Asterism
from asterism.models.ntm import ATLAS, NTM, RSB
from asterism.models.vae import VQAE
from asterism.utils import get_kwargs, kmeans, relabel, to_list
from asterism.utils.acts import Dirichlet
from asterism.utils.data import make_dataset
from asterism.utils.nets import ACTS, NORMS, OPTIMS, Encoder, MLP, RNN
from asterism.utils.plots import show_comparison, show_dataset

In [2]:
from asterism.utils import fib, hello, kmeans

In [3]:
fib(4)

3

In [4]:
hello('Craig')

Hello, Craig!